In [ ]:
%matplotlib widget

import gzip
from dataclasses import dataclass
from functools import partial
from os import listdir, path
from typing import Callable, Optional, Tuple, Union

import msgpack
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.axes import Axes
from matplotlib.collections import PathCollection
from matplotlib.figure import Figure
from scipy import interpolate
from scipy.optimize import curve_fit


In [ ]:
STICKING_PROBABILITIES = [1.0, 0.8, 0.6, 0.4, 0.2, 0.1, 0.0]
DATA_DIR = "/run/media/life/barry/ViennaTools/spherical"


# Function Setup

## Data Loading Routines

In [ ]:
# Some handy type definitions
DataLoadingFunction = Callable[[str], dict]
NumpyOrFloat = Union[float, np.ndarray]


@dataclass(frozen=True, slots=True)
class ThicknessData:
    """Container class for storing extracted layer thickness information"""

    pos: np.ndarray
    thickness: list[list[np.ndarray]]
    sticking_probabilities: list[float]
    max_times: list[int]
    num_points: int


def map_configs(fnames: list[str]) -> dict[str, list[str]]:
    """Takes a list of filenames and splits those into the geometry part as well as the
    sticking probability part. For each geometry all thickness data resulting from
    different sticking probabilities are stored in a list."""
    configs = {}
    for fname in fnames:
        sticking, geom = fname[:5], fname[6:]
        if not geom in configs:
            configs[geom] = []
        configs[geom].append(sticking)
    return configs


def load_data(data_dir: str, filename: str) -> dict:
    """Loads nodal data of .msgpack.gz files storing graph data."""
    with gzip.open(path.join(data_dir, filename)) as gz:
        data = msgpack.unpack(gz, use_list=False, raw=False)
    return data["nodes"]


def extract_data(
    loader: DataLoadingFunction,
    name: str,
    sticking: list[str],
    prefix: str = "extracted_",
) -> ThicknessData:
    """Uses the provided data loader to get the nodal data of a geometry and
    subsequently extracts all fields starting with prefix."""
    data_list: list[list[np.ndarray]] = []
    sticking_probabilities: list[float] = []
    max_times: list[int] = []
    num_points: list[int] = []
    pos = np.array([])
    for ps in sticking:
        node_data = loader(f"{ps}_{name}")
        labels = [s for s in node_data if s.startswith(prefix)]
        data_list.append([np.array(node_data[l]) for l in labels])
        sticking_probabilities.append(float(ps[1:]) / 1000)
        max_times.append(len(labels))
        num_points.extend([d.shape[0] for d in data_list[-1]])
        if len(pos) == 0:
            pos = np.array(node_data["pos"])

    # Ensure that the number of nodes is the same for all data
    assert np.all(np.array(num_points) == num_points[0])

    # Construct the data container instance
    return ThicknessData(
        thickness=data_list,
        sticking_probabilities=sticking_probabilities,
        max_times=max_times,
        pos=pos,
        num_points=num_points[0],
    )


## Data Transformation and Augmentation Routines

In [ ]:
def to_data_tensor(
    thickness_data: ThicknessData,
    sticking_probabilities: list[float],
) -> np.ndarray:
    """Converts the `ThicknessData` instance to a numpy tensor with missing data
    indicated by np.nan. First axis `point_index`, second axis `sticking_probability` and
    third axis `time`."""
    max_time = 50
    data_tensor = np.full(
        (thickness_data.num_points, len(sticking_probabilities), max_time),
        np.nan,
    )
    for i, s in enumerate(sticking_probabilities):
        # First, determine the index
        data_index = 0
        for sp in thickness_data.sticking_probabilities:
            if sp == s:
                break
            data_index = data_index + 1
        else:
            print(f"Couldn't find data for sticking probability {s}")
            continue
        data_tensor[:, i, 0] = 0
        data_tensor[
            :, i, 1 : np.min([thickness_data.max_times[data_index] + 1, max_time])
        ] = np.array(thickness_data.thickness[data_index]).T
    return data_tensor


def fill_conformal(
    data_tensor: np.ndarray, pos: np.ndarray, trench_diameter: float
) -> np.ndarray:
    """Creates a new data array based on the provided data tensor that contains the
    expected deposition radius for p_s=0 (conformal deposition)."""

    expanded_data = data_tensor.copy()
    y = np.where(pos[:, 1] > 0, pos[:, 1], 0)
    time = np.arange(data_tensor.shape[2])

    expanded_data[:, -1, :] = np.outer(
        np.ones(data_tensor.shape[0]),
        time,
    )

    expanded_data[:, -1, :] = expanded_data[:, -1, :] - np.outer(
        y, np.ones(data_tensor.shape[2])
    )

    expanded_data[:, -1, :] = np.where(
        expanded_data[:, -1, :] <= trench_diameter / 2,
        expanded_data[:, -1, :],
        trench_diameter / 2,
    )
    return expanded_data


@dataclass(frozen=True, slots=True)
class GridData:
    xx: np.ndarray
    yy: np.ndarray
    zz: np.ndarray


def to_grid_data(
    z: np.ndarray,
    sticking_probabilities: list[float],
) -> GridData:
    """Converts the provided data to a format compatible with matplotlib's meshgrid used
    by surface plots."""
    y = sticking_probabilities
    x = np.arange(z.shape[1])
    xx, yy = np.meshgrid(x, y)
    # Ensure that all values are >= 0
    zz = np.where(z >= 0, z, np.nan)
    return GridData(xx, yy, zz)


def fill_along_axis(
    grid_data: GridData,
    masking_function: Callable[[np.ndarray], np.ndarray],
    axis: int = 0,
) -> GridData:
    """Fills griddata values for elements where `masking_function`
    evaluates to True using (linear) interpolation along one axis."""
    z = grid_data.zz.copy()
    mask = masking_function(z)
    for (sequence, ma) in zip(np.moveaxis(z, 0, axis), np.moveaxis(mask, 0, axis)):
        if ma.sum() != 0 and (~ma).sum():
            sequence[ma] = np.interp(
                np.flatnonzero(ma), np.flatnonzero(~ma), sequence[~ma]
            )
    return GridData(grid_data.xx, grid_data.yy, z)


def interpolate_data(
    grid_data: GridData,
    sticking_probabilities: list,
    resolution: int = 100,
) -> GridData:
    """Linearly interpolates the provided grid data."""
    xnew = np.linspace(0, grid_data.zz.shape[1], resolution)
    ynew = np.linspace(
        np.min(sticking_probabilities),
        np.max(sticking_probabilities),
        resolution,
    )
    xxnew, yynew = np.meshgrid(xnew, ynew)

    zznew = interpolate.griddata(
        (grid_data.xx.ravel(), grid_data.yy.ravel()),
        grid_data.zz.ravel(),
        (xxnew.ravel(), yynew.ravel()),
        method="linear",
    ).reshape(xxnew.shape)

    return GridData(xxnew, yynew, zznew)


def check_monotonic(
    x: np.ndarray,
    axis: int = 0,
    eps: float = 0.1,
    increasing: bool = True,
):
    """Checks if the provided array is monotonically increasing or decreasing along the
    provided axis"""
    assert axis < len(x.shape)
    if increasing:
        monotonic = np.diff(x, 1, axis=axis, prepend=0) > -eps
    else:
        monotonic = np.diff(x, 1, axis=axis, append=0) > eps
    return monotonic


## Data Visualization Routines

In [ ]:
def plot_data_2d(
    mapped: np.ndarray,
    sticking_probabilities: list[float],
    point_index: int,
    normalize: bool = False,
    log: bool = False,
) -> Tuple[Figure, Axes]:
    """Creates two 2D plots which show the relation of the deposition radius with
    respect to the time dimension as well as the sticking probability"""
    plt.close("all")
    fig, ax = plt.subplots(nrows=1, ncols=2, squeeze=False, figsize=(10, 4))
    cmap = plt.get_cmap("viridis", mapped.shape[2])
    for i, r in enumerate(np.moveaxis(mapped[point_index, :, :], 0, 1)):
        if normalize:
            r = np.where(r != 0, r, np.nan)
            ax[0, 0].plot(
                sticking_probabilities,
                r / (i + 1),
                marker="x",
                color=cmap(i),
            )
        else:
            ax[0, 0].plot(
                sticking_probabilities,
                r,
                marker="x",
                color=cmap(i),
            )

    # Invert the x-axis so that we start with a sticking probability of 1 at the left
    ax[0, 0].invert_xaxis()

    for i, r in enumerate(np.moveaxis(mapped[point_index, :, :], 0, 0)):
        if normalize:
            r = np.where(r != 0, r, np.nan)
            ax[0, 1].plot(
                r / (1 + np.arange(49)),
                marker="x",
                label=sticking_probabilities[i],
            )
        else:
            ax[0, 1].plot(
                r,
                marker="x",
                label=sticking_probabilities[i],
            )

    ax[0, 0].grid(which="both")
    ax[0, 0].set_xlabel("sticking probability")
    ax[0, 0].set_ylabel("radius")
    ax[0, 1].set_xlabel("time")
    ax[0, 1].set_ylabel("radius")
    ax[0, 1].grid(which="both")

    if log:
        ax[0, 0].set_yscale("log")
        ax[0, 1].set_yscale("log")
    ax[0, 1].legend()
    return fig, ax


def draw_3d_plot(
    ax: Axes,
    data: np.ndarray,
    sticking_probabilities: list[float],
    interp: bool = False,
    fill: bool = False,
    monotonic: bool = False,
) -> None:
    """Creates a 3D surface plot of the data in the provided plot axes object."""
    ax.clear()
    grid_data = to_grid_data(data, sticking_probabilities)
    cmap = plt.get_cmap("coolwarm")
    if fill:
        grid_data = fill_along_axis(grid_data, np.isnan, 0)
        if monotonic:
            grid_data = fill_along_axis(
                grid_data, lambda x: ~check_monotonic(x, axis=0), axis=1
            )

    if interp:
        grid_data = interpolate_data(grid_data, sticking_probabilities, resolution=100)

    ax.plot_surface(
        grid_data.xx,
        grid_data.yy,
        grid_data.zz,
        linewidth=1,
        cmap=cmap,
        antialiased=True,
    )

    ax.set_xlabel("time")
    ax.set_ylabel("sticking probability")
    ax.set_zlabel("radius")


def plot_data_3d(
    pos: np.ndarray,
    data_tensor: np.ndarray,
    sticking_probabilities: list[float],
    interp: bool = False,
    fill: bool = False,
    monotonic: bool = False,
    figsize: Tuple[float, float] = (10, 5),
    selection_callback: Optional[Callable[[int, Axes], None]] = None,
) -> Tuple[Figure, Axes]:
    """Plots the geometry and also creates a 3D surface plot of the data corresponding
    to the selected surface point."""

    plt.close("all")

    point_index = 0

    fig = plt.figure(figsize=figsize)
    ax_selector = fig.add_subplot(121)
    ax_3d = fig.add_subplot(122, projection="3d")
    fig.subplots_adjust(hspace=0.4)

    draw_3d_plot(
        ax_3d,
        data_tensor[point_index, :, :],
        sticking_probabilities,
        interp,
        fill,
        monotonic,
    )

    if selection_callback:
        selection_callback(point_index, ax_3d)

    ax_selector.set_title("Geometry\n(selected: 0)")
    ax_selector.scatter(
        pos[:, 0],
        pos[:, 1],
        color="black",
        linewidths=0,
        marker=".",
        picker=True,
        pickradius=0.2,
    )

    scat = ax_selector.scatter(
        pos[point_index, 0], pos[point_index, 1], color="red", marker="o"
    )

    def onpick(event):
        if isinstance(event.artist, PathCollection):
            point_index = event.ind[0]
            colors = np.zeros((len(pos), 3))
            colors[point_index, 0] = 1
            scat.set_offsets((pos[point_index, 0], pos[point_index, 1]))
            ax_selector.set_title(f"Geometry\n(selected: {point_index})")

            # Now redraw 3d plot
            draw_3d_plot(
                ax_3d,
                data_tensor[point_index, :, :],
                sticking_probabilities,
                interp,
                fill,
                monotonic,
                # fit_function,
            )

            if selection_callback:
                selection_callback(point_index, ax_3d)

            fig.canvas.draw_idle()

    ax_selector.axis("scaled")
    fig.canvas.mpl_connect("pick_event", onpick)
    return fig, ax_3d


# Data Exploration

## Loading and Transforming

In [ ]:
# Get all available configurations
fnames = listdir(DATA_DIR)
configs = list(map_configs(fnames).items())

# Select a particular geometry
geometry_index = 13  # 57
name, sticking = configs[geometry_index]

print(name)
trench_diameter = float(name.split("_")[0][1:4])

# Load the data of this geometry
data_loader = partial(load_data, DATA_DIR)

# Load the spherical distribution data (extracted from physical simulations)
extracted = extract_data(data_loader, name, sticking)
data = to_data_tensor(
    extracted,
    STICKING_PROBABILITIES,
)

# Load the viewfactor data
viewfactor_data = to_data_tensor(
    extract_data(data_loader, name, ["s1000"], prefix="viewfactor_"),
    STICKING_PROBABILITIES,
)

# Deviation of the simulation from the viewfactor model
deviation_data = np.zeros_like(data)
for i in range(data.shape[1]):
    deviation_data[:, i, :] = data[:, i, :] - viewfactor_data[:, 0, :]

# Extend data with expected conformal thickness at p_s=0
expanded_data = fill_conformal(data, extracted.pos, trench_diameter)


In [ ]:
def apply_pipeline(pipeline: list, data: np.ndarray) -> np.ndarray:
    for f, args in pipeline:
        data = f(*args)
    return data


pipeline = [
    (fill_along_axis, (np.isnan, 0)),  # Fill NaN
    (
        fill_along_axis,
        (lambda x: ~check_monotonic(x, axis=0), 1),
    ),  # Ensure that data is monotonic
]


## 2D Plots

### Nearest neighbors extracted data

In [ ]:
point_index = 300
fig, _ = plot_data_2d(
    data,
    STICKING_PROBABILITIES,
    point_index=point_index,
    normalize=False,
    log=False,
)


### Viewfactor data

In [ ]:
fig, _ = plot_data_2d(
    viewfactor_data,
    STICKING_PROBABILITIES,
    point_index=point_index,
    normalize=False,
    log=False,
)


### Deviational data

In [ ]:
fig, _ = plot_data_2d(
    deviation_data,
    STICKING_PROBABILITIES,
    point_index=point_index,
    normalize=False,
    log=False,
)


### Extended data (+ conformal thickness)

In [ ]:
fig, _ = plot_data_2d(
    expanded_data,
    STICKING_PROBABILITIES,
    point_index=point_index,
    normalize=False,
    log=False,
)


## 3D Plots

In [ ]:
fig, ax = plot_data_3d(
    extracted.pos,
    data,
    STICKING_PROBABILITIES,
    interp=False,
    fill=True,
    monotonic=False,
)


### Extended dataset (+conformal deposition thickness)

In [ ]:
fig, ax = plot_data_3d(
    extracted.pos,
    expanded_data,
    STICKING_PROBABILITIES,
    interp=False,
    fill=True,
    monotonic=True,
)


## Curve Fitting

In [ ]:
def optimize(
    data: np.ndarray,
    pos: np.ndarray,
    sticking_probabilities: list[float],
    test_function: Callable,
) -> None:
    def fit_surface(
        x: np.ndarray,
        y: np.ndarray,
        a: float,
        b: float,
        c: float,
        d: float,
        e: float,
        f: float,
        g: float,
    ) -> np.ndarray:
        """x: time, y: sp"""
        t = test_function((x, y), a, b, c, d, e, f, g)
        return t

    def callback(point_index: int, ax: Axes) -> None:
        grid_data = to_grid_data(data[point_index, :, :], sticking_probabilities)
        grid_data = fill_along_axis(grid_data, np.isnan, axis=0)
        grid_data = fill_along_axis(
            grid_data, lambda x: ~check_monotonic(x, axis=0), axis=1
        )

        popt, pcov = curve_fit(
            test_function,
            (grid_data.xx.ravel(), grid_data.yy.ravel()),
            grid_data.zz.ravel(),
        )
        print(popt)

        xnew = np.linspace(0, grid_data.zz.shape[1], 100)
        ynew = np.linspace(
            np.min(sticking_probabilities), np.max(sticking_probabilities), 100
        )
        xxnew, yynew = np.meshgrid(xnew, ynew)
        zznew = fit_surface(xxnew.ravel(), yynew.ravel(), *popt)
        ax.plot_surface(
            xxnew,
            yynew,
            zznew.reshape(xxnew.shape),
            linewidth=1,
            antialiased=True,
        )

    plot_data_3d(
        pos,
        data,
        sticking_probabilities,
        interp=False,
        fill=True,
        monotonic=True,
        selection_callback=callback,
    )


def fun(
    X: Union[np.ndarray, Tuple[np.ndarray, np.ndarray]],
    a: float,
    b: float,
    c: float,
    d: float,
    e: float,
    f: float,
    g: float,
) -> np.ndarray:
    # Normalize and reshape the inputs
    x = X[0]  # / 50
    y = 1 - X[1]

    return a * np.tanh(b * x * (c + d * y)) * (e + f * y**2 + g * y**4)


optimize(expanded_data, extracted.pos, STICKING_PROBABILITIES, fun)
